In [ ]:
%load_ext autoreload
%autoreload 2
from bailarn.pos import POSTagger,constant
from bailarn.utils import utils
import numpy as np

# Read Text File

## read raw text 

In [ ]:
# Load tokentizer model

from bailarn.tokenizer import constant as tokenizer_constant
from bailarn.tokenizer.tokenizer import Tokenizer

# Create index for character and tag
char_index = utils.build_tag_index(tokenizer_constant.CHARACTER_LIST, tokenizer_constant.CHAR_START_INDEX)
tag_index = utils.build_tag_index(tokenizer_constant.TAG_LIST, tokenizer_constant.TAG_START_INDEX)

tokenizer_model = Tokenizer(char_index, tag_index)

def tokenize_func(sentence):
    return tokenizer_model.predict(sentence)

In [ ]:
texts = utils.TextCollection(corpus_directory="./data/sample_BEST2010_I2R/", tokenize_function=tokenize_func)
print("Corpus size : {}\n".format(texts.count))
print("Example corpus text : {}\n".format(texts.get_content(0)[0:500]))
print("Tokenized content : {} \n".format(texts.get_token_list(0)[0:20]))

## read pre-tokenize text

In [ ]:
texts = utils.TextCollection(corpus_directory="./data/sample_BEST2010_I2R/", word_delimiter="|",tag_delimiter="/")
print("Corpus size : {}\n".format(texts.count))
print("Example corpus text : {}\n".format(texts.get_content(0)[0:500]))
print("Tokenized content : {} \n".format(texts.get_token_list(0)[0:20]))

# Create word and tag indices

In [ ]:
# Create Word Indexer from vocabs

word_index = utils.build_word_index(texts)

In [ ]:
# Load saved word index

import json

with open('./bailarn/pos/pos_word_index.json', 'r') as f:
    word_index = json.load(f)

In [ ]:
# with open("./bailarn/pos/pos_word_index.json", "w") as write_file:
#     json.dump(word_index, write_file)

In [ ]:
sorted((value,key) for (key,value) in word_index.items())[-10:]

In [ ]:
# Create tag index

tag_index = utils.build_tag_index(constant.TAG_LIST, start_index=1)
tag_index['<PAD>'] = 0

# Transform text into input

In [ ]:
vs = utils.build_input(text_collection=texts, 
                       word_index=word_index, 
                       tag_index=tag_index, 
                       sequence_length=constant.SEQUENCE_LENGTH, 
                       target="pos", 
                       for_train=True)

In [ ]:
print(vs.x[0][:10])

In [ ]:
print(vs.readable_x[0][:10])

In [ ]:
print(vs.y[0][:10])

In [ ]:
print(vs.readable_y[0][:10])

# Train new model without pre-train embedding matrix

In [ ]:
# Default embedding_matrix = None

new_pos_model = POSTagger(new_model=True, tag_index=tag_index)

In [ ]:
new_pos_model.train(vs.x, vs.y, epochs=1, validation_split=0.1, batch_size=64, learning_rate=0.001)

In [ ]:
new_pos_model.predict(vs.x, decode_tag=False)

In [ ]:
new_pos_model.predict(vs.x, decode_tag=True)

In [ ]:
new_pos_model.evaluate(vs.x, vs.y)

In [ ]:
new_pos_model.save('./bailarn/pos/models/new_abc.h5')

# Train new model with pre-train embedding matrix

In [ ]:
# Load w2v model

from bailarn.word_embedder.word2vec import Word2Vec
w2v_model = Word2Vec()

embedding_matrix = utils.get_embedding_matrix(word2vec_model=w2v_model, word_index=word_index)   

In [ ]:
model = POSTagger(embedding_matrix=embedding_matrix, new_model=True, tag_index=tag_index)

In [ ]:
model.train(vs.x, vs.y, epochs=1, validation_split=0.1, batch_size=64, learning_rate=0.001)

In [ ]:
model.evaluate(vs.x, vs.y)

In [ ]:
model.save('./bailarn/pos/models/abc.h5')

# Load pre-train model

In [ ]:
loaded_model = POSTagger(model_path="./bailarn/pos/models/abc.h5", tag_index=tag_index)

In [ ]:
loaded_model.predict(vs.x)

# Use default POS model
- process on pantip data

In [ ]:
# You can manually add text in TextCollection

sample_text = """แจ้งเตือนดีลลวงโลกของNasambkและshopeeต้องอ่านเพื่อไม่เป็นเหยื่ออัพเดตทางshopeeติดต่อมาครับส่วนระบบ
ใครจะถูกผิดผมไม่ทราบได้สิ่งที่ควรทำคือปรับปรุงแก้ไขไม่ว่าจะทางร้านหรือทางShopeeโดยทางshopeeออกโค้ดส่วนลดให้ใหม่
และผมได้เครื่องเรียบร้อยขอบคุณการแก้ไขของshopeeส่วนร้านก็อยากให้ปรับปรุงเพราะผมยังคงกังขาทุกอย่างมันดูแปลกๆแจ้งเตือนดีล
ของร้านNASAmbkในshopeeตามที่ผมคาดการณ์ไว้ยื้อการส่งของจนหมดเวลาระบบshopeeจะยกเลิกออเดอร์อัตโนมัติผมกดขยายเวลาจัดส่ง
ก็ไม่ช่วยอะไรเพราะทางร้านต้นทางไม่กดยอมรับผมได้สั่งซื้อMate00proที่เป็นdealoftheday00,000บาทในวันที่00กพ.
ระบบแจ้งให้ส่งของภายในวันที่00กพ.ปรากฎว่าร้านยื้อไม่ส่งของอ้างสารพัดจนหมดเวลาส่งระบบshopeeจึงทำการยกเลิกออเดอร์อัตโนมัติ
ไม่ได้ของในราคานั้นมันคือขบวนการหลอกลวงต้มตุ๋นนั่นเองดูรายละเอียดในภาพนะครับช่วงแรกคุยกับร้านอีกช่วงคุยกับshopee"""

texts = utils.TextCollection("/", tokenize_function=tokenize_func)
texts.add_text(sample_text)

In [ ]:
text_length = len(texts.get_token_list(0))

In [ ]:
vs = utils.build_input(text_collection=texts,
                        word_index=word_index,
                        tag_index=tag_index,
                        sequence_length=constant.SEQUENCE_LENGTH, # padding size
                        target='pos', for_train=False)

In [ ]:
vs.readable_x[0][:10]

In [ ]:
# Load default model

model = POSTagger(tag_index=tag_index)

In [ ]:
model.predict(vs.x).flatten()[:text_length]